<a href="https://colab.research.google.com/github/nzahin04/pneumonia_classification/blob/main/Capstone_Gradio_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q gradio
import gradio as gr
import requests

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np


In [2]:
from google.colab import files
files.upload()

Saving chest_xray_scan_model.pt to chest_xray_scan_model.pt


In [3]:
model = torchvision.models.resnext101_32x8d(pretrained=True)
saved_model_parameters = torch.load("/content/chest_xray_scan_model.pt", map_location=torch.device('cpu'))

num_features = model.fc.in_features
num_classes = 2

# Replacing model.fc layer
model.fc = torch.nn.Sequential(
           nn.Linear(num_features, 1024),
           nn.Linear(1024, num_classes)
)

model.load_state_dict(saved_model_parameters)

# Placing model in evaluation mode
model.eval()

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [4]:
labels = []
with open('/content/scan_labels (2).txt', 'r') as f:
    for line in f:
        labels.append(line.replace('\n',''))

In [5]:
print(labels)

['NORMAL', 'PNEUMONIA']


In [6]:
input_img = gr.inputs.Image(shape = (224,224), image_mode = 'RGB', source = 'upload', type = 'pil' )

In [7]:
output = gr.outputs.Label(num_top_classes = 2)

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

def classify_image(img):
    img = transform(img).unsqueeze(0)

    with torch.no_grad():
        output = model(img)
    probablilities = torch.nn.functional.softmax(output, dim = 1).flatten()
    return {labels[i]:probablilities[i].item() for i in range(len(labels))}

In [9]:
interface = gr.Interface(fn = classify_image, inputs = input_img, outputs = output)
interface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://44877.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://44877.gradio.app')